## Least squares fit
### Introduction

There are several ways to estimate the slope of the relationship between two variables; the most common is a linear least squares fit. A linear fit is a line intended to model the relationship between variables. A least squares fit is one that minimizes the mean squared error (MSE) between the line and the data.

Suppose we have a sequence of points, `ys`, that we want to express as a function of another sequence `xs`. If there is a linear relationship between xs and ys with intercept inter and slope slope, we expect:

```python
y[i] = inter + slope * x[i]
```

But unless the correlation is perfect, there is a deviation from the line, or residual:

```python
res = ys - (inter + slope * xs)
```

We might try to minimize the absolute value of the residuals, or their squares, or their cubes; but the most common choice is to minimize the sum of squared residuals, `sum(res**2)`, because:

* Squaring treats positive and negative residuals the same

* Squaring gives more weight to large residuals

* If the residuals are uncorrelated and normally distributed with mean 0 and constant (but unknown) variance, then the least squares fit is also the maximum likelihood estimator of inter and slope. See [link](https://en.wikipedia.org/wiki/Linear_regression)

* The values of inter and slope that minimize the squared residuals can be computed efficiently

If you are using xs to predict values of ys, guessing too high might be better (or worse) than guessing too low. In that case you might want to compute some cost function for each residual, and minimize total cost, sum(cost(res)). However, computing a least squares fit is quick, easy and often good enough.

### Implementation

First, we will load a few libraries that we will need for our calculations and plotting:

In [2]:
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt

mpl.rcParams["figure.dpi"] = 150
sns.set_theme()

Then we will load the dataset:

In [19]:
import pandas as pd
import numpy as np
df = pd.read_csv('.lesson/assets/FemPreg.csv')
df.head()

,Unnamed: 0,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,...,laborfor_i,religion_i,metro_i,basewgt,adj_mod_basewgt,finalwgt,secu_p,sest,cmintvw,totalwgt_lb
0,0,1,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,...,0,0,0,3410.389399,3869.349602,6448.271112,2,9,NaN,8.8125
1,1,1,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,...,0,0,0,3410.389399,3869.349602,6448.271112,2,9,NaN,7.8750
2,2,2,1,NaN,NaN,NaN,NaN,5.0,NaN,3.0,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,9.1250
3,3,2,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,7.0000
4,4,2,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,6.1875


And now we can define our own `LeastSquares` function:

In [4]:
def LeastSquares(xs, ys):
    mean_x = np.mean(xs)
    var_x = np.var(xs)
    mean_y = np.mean(ys)
    cov = np.dot(xs - mean_x, ys - mean_y) / len(xs)
    slope = cov / var_x
    inter = mean_y - slope * mean_x
    return inter, slope

`LeastSquares` takes sequences `xs` and `ys` and returns the estimated parameters inter and slope. For details on how it works, see [here](https://en.wikipedia.org/wiki/Numerical_methods_for_linear_least_squares).

We can already apply the function to our dataset, if we first remove the `NAs`.

In [12]:
df = df.dropna(subset=['agepreg', 'totalwgt_lb'])
inter, slope = LeastSquares(df.agepreg, df.totalwgt_lb)
print(f'intercept = {inter}, slope = {slope}')

intercept = 6.830396973311053, slope = 0.017453851471802718


And now that we have the parameters of the model, we define a function to predict:

In [8]:
def FitLine(xs, inter, slope):
    fit_xs = np.sort(xs)
    fit_ys = inter + slope * fit_xs
    return fit_xs, fit_ys

and apply it:

In [14]:
fit_xs, fit_ys = FitLine(df.agepreg, inter, slope)
fit_xs

array([10.83, 10.91, 11.75, ..., 42.75, 43.  , 44.08])

In [15]:
fit_ys

array([7.01942218, 7.02081849, 7.03547973, ..., 7.57654912, 7.58091259,
       7.59976275])

The estimated intercept and slope are $6.8$ lbs and $0.017$ lbs per year. These values are hard to interpret in this form: the intercept is the expected weight of a baby whose mother is 0 years old, which doesn’t make sense in context, and the slope is too small to grasp easily. Let's organize everything in a single dataframe:

In [17]:
df_fit = pd.DataFrame()
df_fit['agepreg'] = fit_xs
df_fit['totalwgt_lb'] = fit_ys
df_fit['type'] = 'fit'

df_train = df.loc[:,['agepreg', 'totalwgt_lb']]
df_train['type'] = 'train'
df_train

,agepreg,totalwgt_lb,type
0,33.16,8.8125,train
1,39.25,7.8750,train
2,14.33,9.1250,train
3,17.83,7.0000,train
4,18.33,6.1875,train
...,...,...,...
13581,30.66,6.3750,train
13584,26.91,6.3750,train
13588,17.91,6.1875,train
13591,21.58,7.5000,train


### Residuals

An important check after performing a linear least squares fit is to calculate the residuals. Residuals takes sequences `xs` and `ys` and estimated parameters `inter` and `slope`. It returns the differences between the actual values and the fitted.

Let's define a function for calculating the residuals:

In [20]:
def Residuals(xs, ys, inter, slope):
    xs = np.asarray(xs)
    ys = np.asarray(ys)
    res = ys - (inter + slope * xs)
    return res

and apply it to our data:

In [21]:
res = Residuals(df_train.agepreg, df_train.totalwgt_lb, inter, slope)
df_res = pd.DataFrame()
df_res['agepreg'] = fit_xs
df_res['totalwgt_lb'] = res
df_res['type'] = 'res'

As mentioned, it is useful to organize everything in the same dataframe:

In [22]:
df_ols = pd.concat([df_fit, df_train, df_res])

For plotting purposes, let's load a few auxiliary functions and created a directory:

In [23]:
from pathlib import Path
Path('plots').mkdir(parents=True, exist_ok=True)
import seaborn as sns
import matplotlib.pyplot as plt

Now we are ready to plot a scatterplot:

In [24]:
scatter1 = sns.relplot(
    data=df_ols,
    x="agepreg",
    y='totalwgt_lb',
    style='type',
    palette='deep',
    hue='type',
    markers={'train':'X', 'fit':'X', 'res':'X'},
    kind='scatter',
    col='type'
    )
scatter1.savefig('plots/scatter.png')
plt.clf()

<Figure size 2385.75x750 with 0 Axes>